### Create a view in databricks

- 1. create a sql file that creates the table as a view
- 2. add it under a job that will run this view. eg the views are under materialized view /prod/analytics/bitx/views and run by this job

NB: When making changins to a view such as deleting or adding or renaming a column, you simply just do it in the view sql file it's self and that's it

In [ ]:
-- #Views
-- ARPU view
CREATE OR REPLACE VIEW bitx_analytics.vw_average_revenue_per_user AS

WITH FirstTransaction AS (
    SELECT user_id, u.preferred_locale, MIN(DATE(transacted_at)) AS first_transaction_date
    FROM experience_analytics.transactions t
    LEFT JOIN bitx_analytics.users_extended u
    ON t.user_id = u.id
    WHERE u.preferred_locale IN ('AU','ZA','GB','FR','ID','MY','NG')
    GROUP BY 1,2
),
DateRange AS ( 
    SELECT user_id, EXPLODE(SEQUENCE(first_transaction_date, date_add(first_transaction_date, 90), interval 1 day)) AS transacted_at
    FROM FirstTransaction
), 
TransactionsWithRevenue AS (
    SELECT 
        t.user_id,
        DATE(t.transacted_at) AS transacted_at_date,
        SUM(t.revenue_usd) AS revenue
    FROM experience_analytics.transactions t
    WHERE t.transaction_name IS NOT NULL
    GROUP BY t.user_id, DATE(t.transacted_at)
),
RevenueWithAllDays AS (
    SELECT 
        dr.user_id,
        ft.preferred_locale,
        ft.first_transaction_date,  
        dr.transacted_at as transacted_at,
        DATEDIFF(dr.transacted_at, ft.first_transaction_date) AS days_since_first_transaction,
        IFNULL(twr.revenue, 0) AS revenue
    FROM DateRange dr
    LEFT JOIN TransactionsWithRevenue twr ON dr.user_id = twr.user_id AND dr.transacted_at = twr.transacted_at_date
    JOIN FirstTransaction ft ON dr.user_id = ft.user_id
),
CumulativeRevenue AS (
    SELECT 
        user_id,
        preferred_locale,
        first_transaction_date,
        transacted_at,
        days_since_first_transaction,
        revenue,
        SUM(revenue) OVER (PARTITION BY user_id ORDER BY transacted_at) AS cumulative_revenue
    FROM RevenueWithAllDays
    WHERE transacted_at <= CURRENT_DATE 
)

SELECT 
    days_since_first_transaction,
    preferred_locale,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-01' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Jan2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-02' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Feb2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-03' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Mar2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-04' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Apr2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-05' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS May2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-06' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Jun2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-07' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Jul2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-08' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Aug2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-09' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Sep2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-10' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Oct2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-11' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Nov2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2023-12' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Dec2023,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-01' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Jan2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-02' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Feb2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-03' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Mar2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-04' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Apr2024, 
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-05' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS May2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-06' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Jun2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-07' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS July2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-08' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Aug2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-09' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Sep2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-10' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Oct2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-11' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Nov2024,
    AVG(CASE WHEN LEFT(first_transaction_date,7) = '2024-12' AND days_since_first_transaction <= DATEDIFF(CURRENT_DATE, first_transaction_date) THEN cumulative_revenue ELSE NULL END) AS Dec2024

FROM CumulativeRevenue
GROUP BY 1,2
ORDER BY days_since_first_transaction, preferred_locale;

ALTER VIEW bitx_analytics.vw_average_revenue_per_user owner TO `super-users`;

In [ ]:
-- #Views
-- Attempted buys view

CREATE OR REPLACE VIEW bitx_analytics.vw_attempted_buys AS

SELECT error,
       SUBSTRING(metadata, CHARINDEX(':', metadata) + 1, LEN(metadata)) AS metadata,
       u.preferred_locale,
       LEFT(attempted_buy_at,7) AS attempted_month,
       COUNT(DISTINCT user_id) AS count_customers,
       COUNT(DISTINCT attempted_buy_at)/COUNT(DISTINCT user_id) AS average_attempted_buys,
       AVG(attempted_amount_usd) AS average_amount_attempted_usd
FROM 
(SELECT d.user_id, 
       b.timestamp_at AS attempted_buy_at,
       d.amount, 
       float(float(b.base_amount) * nullif(t1.average_price_per_usd,0)) AS attempted_amount_usd,
       b.counter_amount,
       d.currency,
       b.error,
       b.metadata
FROM bitx_analytics.deposits_extended d
JOIN broker_analytics.broker_quote_failures b ON d.user_id = b.user_id
LEFT JOIN charter_analytics.currency_converter t1
ON date_trunc('hour', b.timestamp_at) + INTERVAL 15 minutes * ROUND(date_part('minute',  b.timestamp_at) / 15.0) = t1.reference_at
    AND t1.currency = b.base_currency
WHERE d.deposit_number >= 1 
AND d.user_id NOT IN
    (SELECT user_id
    FROM experience_analytics.transactions
    WHERE (transaction_name LIKE 'TradeBuy%' OR transaction_name LIKE 'Buy%')
    )

AND b.timestamp_at >= CURRENT_DATE - INTERVAL '12 months'
) a
LEFT JOIN bitx_analytics.users_extended u
     ON u.id = a.user_id
GROUP BY 1,2,3,4;

ALTER VIEW bitx_analytics.vw_attempted_buys owner TO `super-users`;

In [ ]:
-- #Views
-- Deposits velocity view

CREATE OR REPLACE VIEW bitx_analytics.vw_deposit_velocity (
  user_id COMMENT 'Unique customer identifier',
  created_at COMMENT 'date (day) that deposit was created',
  daily_user_deposit_count COMMENT 'number of deposits on a given day for a given user',
  daily_user_deposit_amount COMMENT 'absolute deposit amount on a given day for a given user',
  daily_user_deposit_amount_usd COMMENT 'total deposit amount in USD on a given day for a given user'
) AS 

WITH deposits_extended_cte AS (
  SELECT
    user_id,
    date_trunc('day', created_at) AS created_at,
    count(*) AS daily_user_deposit_count,
    sum(amount) AS daily_user_deposit_amount,
    sum(amount_usd) AS daily_user_deposit_amount_usd
  FROM
    bitx_analytics.deposits_extended
  GROUP BY 1,2
)
SELECT
  user_id,
  created_at,
  daily_user_deposit_count,
  daily_user_deposit_amount,
  daily_user_deposit_amount_usd
FROM
  deposits_extended_cte;

ALTER VIEW bitx_analytics.vw_deposit_velocity owner TO `super-users`;